In [1]:
import math
import sys
from pathlib import Path
import itertools
import kaleido
import pickle
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.express as px
import plotly.io as pio
from matplotlib import pyplot as plt
from matplotlib_venn import venn3, venn3_circles
from matplotlib_venn import venn2, venn2_circles
from venn import venn
from venn import generate_petal_labels, draw_venn, generate_colors
px.defaults.template = 'ggplot2'
px.defaults.template = 'simple_white'
pio.templates.default = 'simple_white'
cdr_colors = ["#74D2F5", "#5495D6", "#3763F5", "#CF81EB", "#E564F5", "#A04AD6",
    'Aqua', 'AquaMarine']

from collections import namedtuple, defaultdict, Counter

from abag_interactions_hydrophobic import *
from interactions_hydrophobic import *
           
data_dir = Path("/home/pbarletta/labo/22/migue/data")
expdata_dir = Path("/home/pbarletta/labo/22/migue/data/AB-Bind-Database-master")
hydro_dir = Path("/home/pbarletta/labo/22/migue/rtdos/hydro")
aux_dir = Path("/home/pbarletta/labo/22/migue/aux")
pdbs_dir = Path("/home/pbarletta/labo/22/migue/run/pdbs")
mutpdbs_dir = Path("/home/pbarletta/labo/22/migue/run/mut_pdbs")
bin_dir = Path("/home/pbarletta/labo/22/locuaz/bin")
evo_bin = Path(bin_dir, "evoef2/EvoEF2")

AA_LIST = ["ALA", "ARG", "ASN", "ASP", "CYS", "GLU", "GLN", "GLY", "HIS",
           "ILE", "LEU", "LYS", "MET", "PHE", "PRO", "SER", "THR", "TRP", "TYR", "VAL"]
AA_LIST = ("D", "E", "S", "T", "R", "N", "Q", "H", "K", "A", "G", "I", 
    "M", "L", "V", "P", "F", "W", "Y", "C" )

/home/pbarletta/anaconda3/envs/mdtraj/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


------

## Hydrophobic

In [14]:
expdata_df = pd.read_csv(Path(expdata_dir, "AB-Bind_experimental_data.csv"), encoding='latin-1')
pdb_list = tuple(sorted(set(expdata_df['#PDB'])))

abag_chains = {}
for partners, pdb_id in zip(expdata_df["Partners(A_B)"], expdata_df["#PDB"]):
    target, binder = partners.split("_")
    abag_chains[pdb_id] = Chains(antibody=binder, antigen=target) 

mut_list = []
with open(Path(mutpdbs_dir, "mut_list.txt"), "r") as f:
    for mute in f:
        mut_list.append(mute.strip())

pdbs_mut = defaultdict(list)
for mut in mut_list:
    this_pdb = mut.split('-')[0]
    pdbs_mut[this_pdb ].append(mut)

with open(Path.joinpath(hydro_dir, "pdb_hydrophobic.pkl"), "rb") as file:
        pdb_hydrophobic = pickle.load(file)
with open(Path.joinpath(hydro_dir, "mut_hydrophobic.pkl"), "rb") as file:
        mut_hydrophobic = pickle.load(file)

In [19]:
for pdb, mut in pdbs_mut.items():
    pdb_carbons = sum(len(cluster) for cluster in pdb_hydrophobic[pdb])

    mut_carbons = []
    for mut in pdbs_mut[pdb]:
        mut_carbons.append(sum([ len(cluster) for cluster in mut_hydrophobic[mut] ]))
    mean_mut_carbons = np.mean(mut_carbons)

### origin_of_carbons_in_hydrophobic_interactions

---